# Assignment 8

### Grupo 5:
- Adriana Sierra
- Alejandra Navarro
- Elisa Vivar
- Karla Chauca

In [77]:
!pip install rasterstats

### Shapefile

In [78]:
import os
import geopandas as gpd
import rasterio
from rasterstats import zonal_stats
from shapely.ops import transform
import matplotlib.pyplot as plt
import pandas as pd
import pyproj
import numpy as np
import json
import folium as fm

In [79]:
# Load your shapefile
shapefile_path = '../../_data/INEI_LIMITE_DEPARTAMENTAL/INEI_LIMITE_DEPARTAMENTAL.shp'
gdf = gpd.read_file(shapefile_path)
gdf = gdf.to_crs('esri:54009')

### Obtener datos de los rasters

In [80]:
# # Loop

# # Definir los archivos raster y los nombres de los DataFrames
# rasters = {
#         'df10_10':'../../_data/rasters_peru/R10_C10.tif',
#         'df10_11':'../../_data/rasters_peru/R10_C11.tif',
#         'df10_12':'../../_data/rasters_peru/R10_C12.tif',
#         'df11_11':'../../_data/rasters_peru/R11_C11.tif',
#         'df11_12':'../../_data/rasters_peru/R11_C12.tif',
#         'df12_11':'../../_data/rasters_peru/R12_C11.tif',
#         'df12_12':'../../_data/rasters_peru/R12_C12.tif'
# }

# # Diccionario para almacenar los DataFrames
# statistics = {}

# # Bucle para procesar cada archivo raster
# for df_name, raster_path in rasters.items():
#     # Calcular estadísticas zonales
#     stats = zonal_stats(gdf, raster_path, stats="count", categorical=True, all_touched=True)
    
#     # Crear un DataFrame para las estadísticas
#     df_stats = pd.DataFrame(stats)

#     # Cruce con shp
#     df = pd.concat([gdf.reset_index(drop=True), df_stats], axis=1)
    
#     # Almacenar el DataFrame en el diccionario
#     statistics[df_name] = df


In [81]:
# dfs_list = list(statistics.values())

# # Concatenar todos los DataFrames en uno solo
# df_consolidado = pd.concat(dfs_list, ignore_index=True)

# # print(df_consolidado)

### Importar CSV (opcional al loop de los rasters)

In [82]:
# Lee el archivo CSV y carga los datos en un DataFrame
df_consolidado = pd.read_csv('../../_data/group5_ass8_2024_2/output_file.csv')

# Muestra las primeras filas del DataFrame
# print(df_consolidado)

In [83]:
# Modificar la columna de CCDD

def add_leading_zero(num):
    num_str = str(num) 
    if len(num_str) == 1:
        return num_str.zfill(2)  
    return num_str

# Aplicar la función a la columna
df_consolidado['CCDD'] = df_consolidado['CCDD'].apply(add_leading_zero)


In [84]:
df_consolidado['CCDD'] = df_consolidado['CCDD'].astype('object')

### Agrupar para cada departamento

In [85]:
df_f1 = df_consolidado.iloc[:, 14:31] ##seleccionar columnas para trabajar
df_f2 = df_consolidado.iloc[:, [2]]

df_filtered = pd.concat([df_f1, df_f2], axis=1) ##concatena las columnas filtradas
df_filtered

,count,0,1,2,3,4,5,11,12,13,14,21,22,23,24,15,25,CCDD
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,02
2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03
3,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,04
4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170,109331845,108756532.0,26728.0,102935.0,169681.0,28.0,6807.0,259166.0,4326.0,5440.0,NaN,202.0,NaN,NaN,NaN,NaN,NaN,21
171,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22
172,160881397,160264032.0,283956.0,24882.0,8512.0,126.0,23371.0,125106.0,27922.0,118372.0,3077.0,552.0,220.0,1269.0,NaN,NaN,NaN,23
173,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24


In [86]:
df_dep = df_filtered.replace(np.nan, 0 ).groupby('CCDD').sum().reset_index()
df_dep

,CCDD,count,0,1,2,3,4,5,11,12,13,14,21,22,23,24,15,25
0,01,395398036,3.946278e+08,43677.0,89889.0,247525.0,346.0,25140.0,298143.0,15559.0,47990.0,1332.0,114.0,87.0,438.0,36.0,0.0,0.0
1,02,361273310,3.587656e+08,406601.0,201931.0,643838.0,1540.0,60090.0,882292.0,69708.0,224629.0,9348.0,2052.0,777.0,4351.0,236.0,356.0,0.0
2,03,212359714,2.111057e+08,65448.0,128742.0,429504.0,418.0,40435.0,489290.0,33874.0,59269.0,5800.0,1127.0,36.0,83.0,25.0,0.0,0.0
3,04,635523602,6.329721e+08,889781.0,221545.0,206585.0,1038.0,79594.0,581005.0,112333.0,424810.0,19926.0,1755.0,843.0,11681.0,598.0,0.0,0.0
4,05,438022899,4.362867e+08,168915.0,226288.0,476705.0,546.0,48627.0,628270.0,36632.0,133719.0,15510.0,148.0,22.0,781.0,17.0,0.0,0.0
5,06,330630543,3.255923e+08,179228.0,420603.0,1956319.0,754.0,64474.0,2185853.0,64082.0,155690.0,6771.0,1923.0,455.0,2004.0,67.0,0.0,0.0
6,07,1431010,6.037320e+05,367703.0,8467.0,926.0,560.0,47666.0,19778.0,29142.0,288893.0,23283.0,137.0,1638.0,32979.0,6106.0,0.0,0.0
7,08,724005440,7.195472e+08,259390.0,531611.0,1433418.0,2383.0,132466.0,1653138.0,93293.0,247894.0,99517.0,1863.0,62.0,1721.0,523.0,962.0,0.0
8,09,222188324,2.210495e+08,50172.0,137182.0,381275.0,288.0,33281.0,500034.0,9416.0,23065.0,3840.0,219.0,28.0,69.0,0.0,0.0,0.0
9,10,377820981,3.759376e+08,109314.0,196116.0,626282.0,1464.0,54840.0,751569.0,41130.0,90129.0,11663.0,313.0,0.0,238.0,356.0,0.0,0.0


In [87]:
dta_final_depa = gdf[['CCDD', 'NOMBDEP','geometry']].merge(df_dep, on = ['CCDD'], how = 'left').groupby(['CCDD', 'NOMBDEP','geometry']).sum().reset_index()

In [88]:
gdf_depa = gpd.GeoDataFrame(dta_final_depa, geometry='geometry')

### Calcular el área y porcentaje para cada categoría

In [89]:
# Definir la resolución del raster (en metros)
spatial_resolution = 10  # metros
pixel_area = spatial_resolution ** 2  # área de un píxel en m²

# Lista de categorías MSZ específicas
categories = ['1', '2', '3', '4', '5', '11', '12', '13', '14', '15', '21', '22', '23', '24', '25']

# Calcular el área total de la geometría
gdf_depa['geometry_area'] = gdf_depa['geometry'].area

# Calcular el área cubierta y el porcentaje de área cubierta para cada categoría
for category in categories:
    
    category_column = category
    # Calcular el área cubierta por la categoría
    gdf_depa[f'area_c{category}'] = gdf_depa[category_column] * pixel_area
        
    # Calcular el porcentaje de área cubierta por la categoría
    gdf_depa[f'perc_c{category}'] = (gdf_depa[f'area_c{category}'] / gdf_depa['geometry_area']) * 100

# Imprimir el DataFrame con los nuevos porcentajes de cobertura
#print(gdf_depa[['geometry_area'] + [f'area_c{category}' for category in categories] + [f'perc_c{category}' for category in categories]])

print(gdf_depa)

   CCDD        NOMBDEP                                           geometry  \
0    01       AMAZONAS  POLYGON ((-7792169.198 -369977.341, -7792239.4...   
1    02         ANCASH  POLYGON ((-7734578.955 -994191.696, -7734565.2...   
2    03       APURIMAC  POLYGON ((-7270386.011 -1623441.123, -7270292....   
3    04       AREQUIPA  POLYGON ((-7068605.941 -1802675.352, -7068565....   
4    05       AYACUCHO  POLYGON ((-7347505.611 -1501033.916, -7347646....   
5    06      CAJAMARCA  POLYGON ((-7795192.034 -958786.093, -7795243.4...   
6    07         CALLAO  POLYGON ((-7629126.935 -1457284.548, -7629105....   
7    08          CUSCO  POLYGON ((-7226452.405 -1387808.831, -7226306....   
8    09   HUANCAVELICA  POLYGON ((-7372714.774 -1477995.808, -7372676....   
9    10        HUANUCO  POLYGON ((-7566554.467 -1027471.984, -7566363....   
10   11            ICA  POLYGON ((-7458111.182 -1597988.159, -7458072....   
11   12          JUNIN  POLYGON ((-7439578.366 -1317878.904, -7439448....   

### Mapas

In [90]:
# ### Solo para una categoría

# # Coordenadoas de Palacio de Gobierno. Centro de Lima como el centro
# lat_palacio = -12.0757538
# long_palacio = -76.9863174
# zoom_start = 5

# z = fm.Map(location = [lat_palacio, long_palacio], tiles='cartodbpositron', zoom_start = zoom_start)

# fm.Choropleth(
#     geo_data=gdf, 
#     data=gdf_depa, 
#     columns=['CCDD', 'area_c21'], 
#     key_on="feature.properties.CCDD",
#     fill_color="YlOrRd", 
#     fill_opacity=0.8, 
#     line_opacity=0.2, 
#     legend_name="MSZ Category Coverage (%)", 
#     smooth_factor=0,
#     Highlight= True, 
#     line_color = "#0000", 
#     overlay=True, 
#     nan_fill_color = "White"  
#     ).add_to(z)

# fm.LayerControl().add_to(z)

# # Save in a html format 
# # z.save("MSZ_Map1.html")

# z

In [91]:
# ### Usando layers para todas las categorías

# map_center = [-12.0757538, -76.9863174]
# mymap = fm.Map(location=map_center, zoom_start=5)

# # Crear y agregar un Choropleth para cada categoría al mapa
# for category in categories:
#     layer_name = f'perc_c{category}'
    
#     # Crear una capa de Choropleth
#     choropleth = fm.Choropleth(
#         geo_data=gdf,
#         data=gdf_depa,
#         columns=['CCDD', layer_name],
#         key_on='feature.properties.CCDD',
#         fill_color='YlOrRd',  
#         fill_opacity=0.7,
#         line_opacity=0.2,
#         legend_name=f'MSZ {category} Coverage'
#     )
    
#     # Agregar la capa Choropleth al mapa
#     choropleth.add_to(mymap)

# # Agregar el control de capas al mapa
# fm.LayerControl().add_to(mymap)

# # Guardar el mapa a un archivo HTML
# #mymap.save("MSZ_Map2.html")

# mymap